<a href="https://colab.research.google.com/github/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/blob/feature%2Fschools-data%2Fade-ade/notebooks/merge_csvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
db_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/final_csv/Final_with_gov_ID.csv")
harvest_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/share_of_harvest_sold.csv")

In [ ]:
print(db_df.shape)
print(harvest_df.shape)

(4483, 20)
(30, 3)


In [5]:
db_df.drop(['Unnamed: 0','Cell','Form: Form Name','Cell_ID'],axis=1,inplace=True)
harvest_df.drop('Unnamed: 0', axis=1,inplace=True)

In [6]:
# Merge database file on share of harvest in each RW district
df = pd.merge(db_df,harvest_df,on='District',how='inner')

In [ ]:
# db_df['District'].nunique()
# df['District'].nunique()
# harvest_df['Mean share of harvest sold'].nunique()
# df['Mean share of harvest sold'].nunique()

In [7]:
schools_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/Rwanda_Schools%20(1).csv")

In [8]:
govt_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/unique_gov_id_PDS.csv')

In [9]:
govt_df['Sector'] = govt_df['Sector'].str.title()
schools_df['Sector'] = schools_df['Sector'].str.title()
schools_df['District'] = schools_df['District'].str.title()

In [10]:
# Correct spellings in schools data to match govt_data 
govt_to_sch = {
    'Kabagali' : 'Kabagari',
    'Muringa' : 'Mulinga' ,
    'Rugarika' : 'Rugalika',
    'Gishali' : 'Gishari' ,
    'Rugengabari' : 'Rugengabali',
    'Nyagihanga':'Nyagahanga',
    'Niboye': 'Niboyi',
    'Save':'Gatoki',
    'Tumba':'Ntuba'
    }

for key,value in govt_to_sch.items():

  schools_df.loc[schools_df['Sector'] == value,'Sector'] = key

In [ ]:
b = [x for x in set(schools_df['Sector']) if x not in set(govt_df['Sector'])]
b

[]

In [11]:
# Change Nyanza-Kibirizi to Nyanza-Kibilizi to match database_df
indy = schools_df[(schools_df['District'] == "Nyanza") & (schools_df['Sector'] == 'Kibirizi')].index

for i in indy:
  schools_df.loc[i,['Sector']] = "Kibilizi"

In [12]:
schools_df.drop('Unnamed: 0',axis=1,inplace=True)

In [15]:
# Correct Sector names in schools data 
il = [294]

for i in il:
  schools_df.loc[i,['Sector']] = 'Bigogwe'

jl = [185]

for j in jl:
  schools_df.loc[j,['Sector']] = 'Ngarama'

In [16]:
# Merge schools data on govt data 
mg = pd.merge(schools_df,govt_df,on=['District','Sector'])

In [ ]:
y = [x for x in set(mg['School Name']) if x not in set(schools_df['School Name'])]
y

[]

In [ ]:
q = [x for x in set(schools_df['School Name']) if x not in set(mg['School Name'])]
q

[]

In [ ]:
schools_df[schools_df['School Name'] == 'College Baptiste De Ngarama']

,School Name,Province,District,Sector
185,College Baptiste De Ngarama,East,Gatsibo,Ngarama


In [ ]:
mg[mg['School Name'] == 'EAV BIGOGWE']

,School Name,Province_x,District,Sector,Prov_ID,Province_y,Dist_ID,Sect_ID
411,EAV BIGOGWE,West,Nyabihu,Bigogwe,3,Iburengerazuba,34,3401


In [17]:
mg[mg['School Name'] == 'College Baptiste De Ngarama']

,School Name,Province_x,District,Sector,Prov_ID,Province_y,Dist_ID,Sect_ID
1212,College Baptiste De Ngarama,East,Gatsibo,Ngarama,5,Iburasirazuba,53,5310


In [ ]:
# schools_df.groupby(['District','Sector'],as_index=False).count()

In [18]:
# Get the count of schools in each sector 
count_sch = mg.groupby(['Sect_ID']).count()['School Name']
count_sch.head(3)

Sect_ID
1101    2
1102    2
1103    3
Name: School Name, dtype: int64

In [35]:
# merge database df on school count 
test_df = pd.merge(df,count_sch,on='Sect_ID')

In [38]:
test_df['School_count'] = test_df['School Name']

In [39]:
test_df.drop('School Name',axis=1,inplace=True)

In [40]:
current_df = test_df

In [42]:
# Current final df with farm harvest by District 
# and number of schools in each Sector
current_df.shape

(4483, 18)